In [1]:
# Скачивание датасета
!curl https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz --output data/flowers.tgz
!tar -xvzf data/flowers.tgz -C data/ -h

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  25.0M      0  0:00:08  0:00:08 --:--:-- 28.3M
tar: Option -L is not permitted in mode -x


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import copy
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = 'data/flower_photos'
model_path = 'best_vgg16_flowers.pth'
num_epochs = 10
num_classes = 5
learning_rate=0.001
best_accuracy = 0.0

### Train model

In [3]:
# Преобразования данных (например, изменение размера и нормализация)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_split = 0.8
image_datasets = datasets.ImageFolder(data_dir, transform=data_transforms['train'])
train_size = int(train_split * len(image_datasets))
val_size = len(image_datasets) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(image_datasets, [train_size, val_size])

# Создаем DataLoader'ы для обучения и валидации
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True)
}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
num_classes = len(image_datasets.classes)

# Загрузка предобученной модели VGG-16
model = models.vgg16(pretrained=True)

# Заморозим все слои, кроме последнего
for param in model.parameters():
    param.requires_grad = False

# Изменим последний классификатор под количество классов в нашем датасете
model.classifier[6] = nn.Linear(4096, num_classes)
model = model.to(device)

# Определяем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=learning_rate)

# Функция для обучения модели
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
    
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Включаем режим обучения
            else:
                model.eval()   # Включаем режим оценки
    
            running_loss = 0.0
            running_corrects = 0
    
            # Проходим по батчам данных
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
    
                # Обнуляем градиенты
                optimizer.zero_grad()
    
                # Прямой проход
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
    
                    # Обратное распространение и оптимизация только на обучении
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
    
                # Статистика
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
    
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Обучаем модель
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)

# Сохраняем модель в файл
torch.save(model.state_dict(), model_path)



/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/9
----------
train Loss: 0.6318 Acc: 0.7606
val Loss: 0.4382 Acc: 0.8406
Epoch 1/9
----------
train Loss: 0.4611 Acc: 0.8297
val Loss: 0.4226 Acc: 0.8515
Epoch 2/9
----------
train Loss: 0.4093 Acc: 0.8495
val Loss: 0.4342 Acc: 0.8392
Epoch 3/9
----------
train Loss: 0.4084 Acc: 0.8501
val Loss: 0.4269 Acc: 0.8542
Epoch 4/9
----------
train Loss: 0.3859 Acc: 0.8597
val Loss: 0.4100 Acc: 0.8624
Epoch 5/9
----------
train Loss: 0.3709 Acc: 0.8699
val Loss: 0.4009 Acc: 0.8638
Epoch 6/9
----------
train Loss: 0.3569 Acc: 0.8726
val Loss: 0.3984 Acc: 0.8638
Epoch 7/9
----------
train Loss: 0.3687 Acc: 0.8600
val Loss: 0.3931 Acc: 0.8733
Epoch 8/9
----------
train Loss: 0.3569 Acc: 0.8678
val Loss: 0.3918 Acc: 0.8624
Epoch 9/9
----------
train Loss: 0.3850 Acc: 0.8566
val Loss: 0.3962 Acc: 0.8747


### Inference model

In [4]:
model = models.vgg16(pretrained=False)  # Создаем модель VGG-16 без предобученных весов
model.classifier[6] = nn.Linear(4096, num_classes)  # Меняем последний классификационный слой
model.load_state_dict(torch.load(model_path)) # Загружаем обученные веса
model = model.to(device)
model.eval()

/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/var/folders/9b/cz5kycc54pq7926rs5ddp6t00000gp/T/ipykernel_70488/3127508217.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allo

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
%%time

# Функция для выполнения инференса
def inference(model, inputs):
    inputs = inputs.to(device)
    with torch.no_grad():  # Отключаем градиенты для инференса
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    return preds.cpu().numpy()  # Возвращаем предсказания в формате numpy

# Оценка на валидационном наборе данных
correct = 0
total = 0

for inputs, labels in dataloaders['val']:
    # Получаем предсказания модели
    preds = inference(model, inputs)

    # Сравнение предсказанных классов с истинными
    correct += np.sum(preds == labels.numpy())
    total += labels.size(0)

# Выводим точность модели
accuracy = correct / total
print(f'Accuracy on validation dataset: {accuracy:.4f}')

Accuracy on validation dataset: 0.8706
CPU times: user 11min 13s, sys: 1min 38s, total: 12min 52s
Wall time: 3min 13s


### Save model to ONXX

In [6]:
model = models.vgg16(pretrained=False)  # Создаем модель VGG-16 без предобученных весов
model.classifier[6] = torch.nn.Linear(4096, num_classes)  # Меняем последний классификационный слой
model.load_state_dict(torch.load(model_path)) # Загружаем обученные веса
model = model.to(device)
model.eval()  # Устанавливаем режим оценки

# Создаем фиктивный входной тензор (batch size 1, 3 канала, 224x224 пикселей)
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Путь для сохранения ONNX модели
onnx_model_path = 'vgg16_flowers.onnx'

# Экспортируем модель в формат ONNX
torch.onnx.export(
    model,                     # Модель для экспорта
    dummy_input,               # Пример входного тензора
    onnx_model_path,           # Имя выходного файла
    export_params=True,        # Экспортируем обученные параметры
    opset_version=11,          # Версия ONNX opset
    do_constant_folding=True,  # Оптимизация неизменяемых частей
    input_names=['input'],     # Имя входного тензора
    output_names=['output'],   # Имя выходного тензора
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})  # Динамическое изменение размера батча

print(f"Модель VGG-16 успешно экспортирована в {onnx_model_path}")


/var/folders/9b/cz5kycc54pq7926rs5ddp6t00000gp/T/ipykernel_70488/2472523256.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))

Модель VGG-16 успешно экспортирована в vgg16_flowers.onnx


### Inference model in ONXX

In [7]:
import onnxruntime as ort

ort_session = ort.InferenceSession(onnx_model_path)

In [8]:
%%time

# Функция для инференса
def onnx_inference(ort_session, inputs):
    ort_inputs = {ort_session.get_inputs()[0].name: inputs}
    ort_outs = ort_session.run(None, ort_inputs)
    return ort_outs

# Оценка на валидационном наборе данных
correct = 0
total = 0

for inputs, labels in dataloaders['val']:
    inputs_numpy = inputs.numpy()
    outputs = onnx_inference(ort_session, inputs_numpy)
    preds = np.argmax(outputs[0], axis=1)
    correct += np.sum(preds == labels.numpy())
    total += labels.size(0)

# Выводим точность модели
accuracy = correct / total
print(f'Accuracy on validation dataset: {accuracy:.4f}')

Accuracy on validation dataset: 0.8610
CPU times: user 14min 34s, sys: 1min, total: 15min 35s
Wall time: 4min 25s


### Save model in tensorflow ONXX

In [9]:
from onnx_tf.backend import prepare
import onnx

# Загружаем модель в формате ONNX
onnx_model = onnx.load(onnx_model_path)

# Конвертируем ONNX модель в TensorFlow формат
tf_rep = prepare(onnx_model)
tf_model_path = 'vgg16_flowers_tf_model'
tf_rep.export_graph(tf_model_path)
print(f"Модель экспортирована в TensorFlow формат: {tf_model_path}")


/Users/aleksandr/Desktop/micro_llm/venv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_53_x, mul_3_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: vgg16_flowers_tf_model/assets


INFO:tensorflow:Assets written to: vgg16_flowers_tf_model/assets
INFO:absl:Writing fingerprint to vgg16_flowers_tf_model/fingerprint.pb


Модель экспортирована в TensorFlow формат: vgg16_flowers_tf_model


### Inference model in tensorflow ONXX

In [10]:
import tensorflow as tf

# Загрузка модели TensorFlow
model = tf.saved_model.load(tf_model_path)

In [11]:
%%time

# Выполняем инференс
def tensorflow_inference(model, inputs):
    inputs_tf = tf.convert_to_tensor(inputs, dtype=tf.float32)
    outputs = model.signatures['serving_default'](inputs_tf)['output']
    return outputs

correct = 0
total = 0
for inputs, labels in dataloaders['val']:
    inputs_numpy = inputs.numpy()
    outputs = tensorflow_inference(model, inputs_numpy)
    preds = np.argmax(outputs, axis=1)
    correct += np.sum(preds == labels.numpy())
    total += labels.size(0)

# Выводим точность модели
accuracy = correct / total
print(f'Accuracy on validation dataset: {accuracy:.4f}')

Accuracy on validation dataset: 0.8569
CPU times: user 17min 53s, sys: 1min 49s, total: 19min 42s
Wall time: 3min 55s


In [14]:
import torch.nn.utils.prune as prune

# Функция для применения обрезки к модели
def prune_model(model, pruning_percentage):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):  # Применяем обрезку только к слоям nn.Linear
            prune.l1_unstructured(module, name='weight', amount=pruning_percentage)
    return model

# Функция для оценки модели
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    for inputs, labels in dataloaders['val']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels.data)
        total += labels.size(0)
    accuracy = correct.double() / total
    return accuracy.item()

# Загружаем сохраненные веса модели
pruning_percentages = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # Уровни разреженности
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, num_classes)
model.load_state_dict(torch.load(model_path))
model = model.to(device)
# Оценка исходной модели до обрезки
initial_accuracy = evaluate_model(model)
print(f'Accuracy before pruning: {initial_accuracy:.4f}')

# Применяем обрезку и оцениваем модель на разных уровнях sparsity
for pruning_percentage in pruning_percentages:
    pruned_model = prune_model(model, pruning_percentage)
    pruned_accuracy = evaluate_model(pruned_model)
    print(f'Pruning {pruning_percentage * 100}% - Accuracy: {pruned_accuracy:.4f}')
    if pruned_accuracy < 0.4:
        print(f'Training accuracy fell below 40% at {pruning_percentage * 100}% sparsity.')
        # break


/var/folders/9b/cz5kycc54pq7926rs5ddp6t00000gp/T/ipykernel_70488/1224870059.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)

Accuracy before pruning: 0.8638
Pruning 10.0% - Accuracy: 0.8556
Pruning 20.0% - Accuracy: 0.8460
Pruning 30.0% - Accuracy: 0.8542
Pruning 40.0% - Accuracy: 0.8597
Pruning 50.0% - Accuracy: 0.8065
Pruning 60.0% - Accuracy: 0.5845
Pruning 70.0% - Accuracy: 0.2657
Training accuracy fell below 40% at 70.0% sparsity.
Pruning 80.0% - Accuracy: 0.2207
Training accuracy fell below 40% at 80.0% sparsity.
Pruning 90.0% - Accuracy: 0.2534
Training accuracy fell below 40% at 90.0% sparsity.


Pruning 10% - Accuracy: 0.85 — на начальном этапе точность даже немного улучшилась после обрезки. Это может объясняться тем, что небольшая разреженность помогает модели избавиться от менее значимых весов, что предотвращает переобучение.
Pruning 20% - Accuracy: 0.86 и Pruning 30% - Accuracy: 0.85 и Pruning 40% - Accuracy: 0.85 —  Модель теряет часть весов, но все еще способна делать хорошие предсказания.
Pruning 40% - Accuracy: 0.85 — точность остается стабильной. До этого порога модель сохраняет свою структуру, несмотря на обрезку до 40%. </br>
Pruning 50% - Accuracy: 0.82 и Pruning 60% - Accuracy: 0.56 — с увеличением уровня разреженности точность начинает заметно снижаться, так как важные параметры начинают обнуляться, что ухудшает способность модели делать точные предсказания.
Pruning 70% - Accuracy: 0.25 — модель практически теряет способность правильно классифицировать объекты. </br>
_Точность обучения падает ниже 40% при разреженности 70%, что делает модель практически бесполезной._


In [17]:
# Загрузка предобученной модели VGG-16
model = models.vgg16(pretrained=True)

# Настройка данных
data_dir = 'data/flower_photos'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = 'best_vgg16_flowers.pth'
num_classes = 5
learning_rate = 0.001
num_epochs = 10

# Преобразования данных
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Загрузка датасета и разделение на тренировочные и валидационные данные
train_split = 0.8
image_datasets = datasets.ImageFolder(data_dir, transform=data_transforms['train'])
train_size = int(train_split * len(image_datasets))
val_size = len(image_datasets) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(image_datasets, [train_size, val_size])

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True)
}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
for param in model.parameters():
    param.requires_grad = False
model.classifier[6] = nn.Linear(4096, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=learning_rate)

# Функция для оценки модели
def evaluate_model(model, dataloaders):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            total += labels.size(0)
    return correct.double() / total


layer_sparsity = {}
model.load_state_dict(torch.load(model_path))
initial_accuracy = evaluate_model(model, dataloaders)
target_accuracy = initial_accuracy * 0.9

print(f'Initial accuracy: {initial_accuracy:.4f}, Target accuracy: {target_accuracy:.4f}')


pruned_model = copy.deepcopy(model)

# Применение prunining для каждого слоя
for name, module in pruned_model.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        current_sparsity = 0.0
        layer_accuracy = initial_accuracy
        
        while layer_accuracy >= target_accuracy:
            current_sparsity += 0.05  # Увеличиваем sparsity на 5% за шаг
            
            # Применяем прунинг к весам текущего слоя
            prune.ln_structured(module, name='weight', amount=current_sparsity, n=2, dim=0)
            layer_accuracy = evaluate_model(pruned_model, dataloaders)
            print(f'Layer: {name}, Sparsity: {current_sparsity*100:.2f}%, Accuracy: {layer_accuracy:.4f}')
            if layer_accuracy < target_accuracy:
                # Если точность упала ниже целевого значения, остановка прунинга этого слоя
                layer_sparsity[name] = current_sparsity - 0.05  # Сохраняем последнее валидное значение sparsity
                break

print("Sparsity values for each layer:")
for layer, sparsity in layer_sparsity.items():
    print(f'{layer}: {sparsity*100:.2f}%')


/var/folders/9b/cz5kycc54pq7926rs5ddp6t00000gp/T/ipykernel_70488/3386099547.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)

Initial accuracy: 0.9005, Target accuracy: 0.8105
Layer: features.0, Sparsity: 5.00%, Accuracy: 0.9033
Layer: features.0, Sparsity: 10.00%, Accuracy: 0.8965
Layer: features.0, Sparsity: 15.00%, Accuracy: 0.8896
Layer: features.0, Sparsity: 20.00%, Accuracy: 0.8597
Layer: features.0, Sparsity: 25.00%, Accuracy: 0.7929
Layer: features.2, Sparsity: 5.00%, Accuracy: 0.7752
Layer: features.5, Sparsity: 5.00%, Accuracy: 0.7861
Layer: features.7, Sparsity: 5.00%, Accuracy: 0.7807
Layer: features.10, Sparsity: 5.00%, Accuracy: 0.7861
Layer: features.12, Sparsity: 5.00%, Accuracy: 0.7847
Layer: features.14, Sparsity: 5.00%, Accuracy: 0.7316
Layer: features.17, Sparsity: 5.00%, Accuracy: 0.7234
Layer: features.19, Sparsity: 5.00%, Accuracy: 0.7262
Layer: features.21, Sparsity: 5.00%, Accuracy: 0.6921
Layer: features.24, Sparsity: 5.00%, Accuracy: 0.7057
Layer: features.26, Sparsity: 5.00%, Accuracy: 0.6866
Layer: features.28, Sparsity: 5.00%, Accuracy: 0.6635
Layer: classifier.0, Sparsity: 5.00%

Прунинг был эффективен с уменьшением точности до нужного уровня при sparsity в 20%. </br>
Для слоев начиная с features.2 и до конца прунинг не был применен, так как при первых тестах sparsity в 5% сразу значительно снижало точность ниже целевого порога (0.8105).  </br>

In [18]:
import torch.nn.utils.prune as prune
import torch.optim as optim

# Функция для применения обрезки к модели
def prune_model(model, pruning_percentage):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):  # Применяем обрезку только к слоям nn.Linear
            prune.l1_unstructured(module, name='weight', amount=pruning_percentage)
    return model

# Функция для оценки модели
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    for inputs, labels in dataloaders['val']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels.data)
        total += labels.size(0)
    accuracy = correct.double() / total
    return accuracy.item()

# Функция для дообучения модели после обрезки
def fine_tune_model(model, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        total = 0
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total

        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

    return model

# Загрузка модели и ее подготовка к прунингу
pruning_percentages = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # Уровни разреженности
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, num_classes)
model.load_state_dict(torch.load(model_path))
model = model.to(device)

# Оценка исходной модели до обрезки
initial_accuracy = evaluate_model(model)
print(f'Accuracy before pruning: {initial_accuracy:.4f}')

# Определение функции потерь и оптимизатора для дообучения
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Применяем обрезку, дообучаем и оцениваем модель на разных уровнях sparsity
for pruning_percentage in pruning_percentages:
    print(f'\nApplying {pruning_percentage * 100}% pruning...')
    pruned_model = prune_model(model, pruning_percentage)

    # Размораживаем параметры модели для дообучения
    for param in pruned_model.parameters():
        param.requires_grad = True

    # Дообучаем модель после прунинга
    pruned_model = fine_tune_model(pruned_model, criterion, optimizer, num_epochs=3)
    pruned_accuracy = evaluate_model(pruned_model)
    print(f'Pruning {pruning_percentage * 100}% - Accuracy after fine-tuning: {pruned_accuracy:.4f}')
    if pruned_accuracy < 0.4:
        print(f'Training accuracy fell below 40% at {pruning_percentage * 100}% sparsity.')
        break


/var/folders/9b/cz5kycc54pq7926rs5ddp6t00000gp/T/ipykernel_70488/2311144916.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)

Accuracy before pruning: 0.8896

Applying 10.0% pruning...
Epoch 0/2, Loss: 1.9222, Accuracy: 0.3147
Epoch 1/2, Loss: 1.3048, Accuracy: 0.4292
Epoch 2/2, Loss: 1.2551, Accuracy: 0.4707
Pruning 10.0% - Accuracy after fine-tuning: 0.4728

Applying 20.0% pruning...
Epoch 0/2, Loss: 1.2211, Accuracy: 0.4772
Epoch 1/2, Loss: 1.2112, Accuracy: 0.4922
Epoch 2/2, Loss: 1.1675, Accuracy: 0.5037
Pruning 20.0% - Accuracy after fine-tuning: 0.5313

Applying 30.0% pruning...
Epoch 0/2, Loss: 1.1834, Accuracy: 0.5082
Epoch 1/2, Loss: 1.0990, Accuracy: 0.5589
Epoch 2/2, Loss: 1.1205, Accuracy: 0.5545
Pruning 30.0% - Accuracy after fine-tuning: 0.5381

Applying 40.0% pruning...
Epoch 0/2, Loss: 1.1186, Accuracy: 0.5518
Epoch 1/2, Loss: 1.0865, Accuracy: 0.5671
Epoch 2/2, Loss: 1.0702, Accuracy: 0.5777
Pruning 40.0% - Accuracy after fine-tuning: 0.5627

Applying 50.0% pruning...
Epoch 0/2, Loss: 1.0566, Accuracy: 0.5916
Epoch 1/2, Loss: 1.0541, Accuracy: 0.5807
Epoch 2/2, Loss: 1.0376, Accuracy: 0.5834

Удалось достичь максимального уровня разреженности в **70%** с сохранением приемлемого качества модели. При 70% sparsity точность модели после fine-tuning составила **63.49%**, что является наивысшим значением среди всех уровней sparsity до 80%. </br>

Однако, начиная с **80%** sparsity, точность начала снижаться, и при 90% sparsity точность модели резко упала до **33.24%**, что значительно ниже требуемого порога в 40%. </br>